In [1]:
import networkx as nx
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
allgenes = pd.read_csv('data/all_genes.txt', sep='\n', header=[0])
mappings = df = pd.DataFrame(pd.read_excel("data/mapping.xlsx"))
# print(mappings.shape)
# print(mappings.head())
mappings=pd.merge(allgenes, mappings, on="Entrenz")
# print(mappings.head())
# print(mappings.shape)
p_list = mappings['Symbol'].tolist()
# protein_list=['TPH1','COMT','SLC18A2','HTR1B','HTR2C','HTR2A','MAOA',
#             'TPH2','HTR1A','HTR7','SLC6A4','GABBR2','POMC','GNAI3',
#             'NPY','ADCY1','PDYN','GRM2','GRM3','GABBR1']

In [3]:
lines=[]
for i in range(0,len(p_list),20):
    protein_list=p_list[i:i+20]
    proteins = '%0d'.join(protein_list)
    # print(protein_list)
    url = 'https://string-db.org/api/tsv/network?identifiers=' + proteins + '&species=9606'
    r = requests.get(url)
    if(r.status_code==200):
        lines.extend(r.text.split('\n')) # pull the text from the response object and split based on new lines
# print(lines)
data = [l.split('\t') for l in lines] # split each line into its components based on tabs
# convert to dataframe using the first row as the column names; drop empty, final row
df = pd.DataFrame(data[1:-1], columns = data[0]) 
# dataframe with the preferred names of the two proteins and the score of the interaction
interactions = df[['preferredName_A', 'preferredName_B', 'score']]


In [5]:
interactions=interactions.dropna()
print(interactions.head())
print(interactions.shape)
# mappings.columns = mappings.iloc[0] 
print(mappings.shape)
for ind1, row1 in mappings.iterrows():
    for index, row in interactions.iterrows():
        if(row['preferredName_A'].lower()==row1['Symbol'].lower()):
            row['preferredName_A']=row1['Entrenz']
        if(row['preferredName_B'].lower()==row1['Symbol'].lower()):
            row['preferredName_B']=row1['Entrenz']
    print(ind1,end="")




  preferredName_A preferredName_B  score
0            CDH3            CDH4  0.569
1            CDH3            CDH4  0.569
2            CDH3            CDH2  0.575
3            CDH3            CDH2  0.575
4           PDCD6         PDCD6IP  0.998
(6199, 3)
(5186, 2)


0123456789101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293949596979899100101102103104105106107108109110111112113114115116117118119120121122123124125126127128129130131132133134135136137138139140141142143144145146147148149150151152153154155156157158159160161162163164165166167168169170171172173174175176177178179180181182183184185186187188189190191192193194195196197198199200201202203204205206207208209210211212213214215216217218219220221222223224225226227228229230231232233234235236237238239240241242243244245246247248249250251252253254255256257258259260261262263264265266267268269270271272273274275276277278279280281282283284285286287288289290291292293294295296297298299300301302303304305306307308309310311312313314315316317318319320321322323324325326327328329330331332333334335336337338339340341342343344345346347348349350351352353354355356357358359360361362363364365366367368369

2326232723282329233023312332233323342335233623372338233923402341234223432344234523462347234823492350235123522353235423552356235723582359236023612362236323642365236623672368236923702371237223732374237523762377237823792380238123822383238423852386238723882389239023912392239323942395239623972398239924002401240224032404240524062407240824092410241124122413241424152416241724182419242024212422242324242425242624272428242924302431243224332434243524362437243824392440244124422443244424452446244724482449245024512452245324542455245624572458245924602461246224632464246524662467246824692470247124722473247424752476247724782479248024812482248324842485248624872488248924902491249224932494249524962497249824992500250125022503250425052506250725082509251025112512251325142515251625172518251925202521252225232524252525262527252825292530253125322533253425352536253725382539254025412542254325442545254625472548254925502551255225532554255525562557255825592560256125622563256425652566256725682569257025712572257325742575

4375437643774378437943804381438243834384438543864387438843894390439143924393439443954396439743984399440044014402440344044405440644074408440944104411441244134414441544164417441844194420442144224423442444254426442744284429443044314432443344344435443644374438443944404441444244434444444544464447444844494450445144524453445444554456445744584459446044614462446344644465446644674468446944704471447244734474447544764477447844794480448144824483448444854486448744884489449044914492449344944495449644974498449945004501450245034504450545064507450845094510451145124513451445154516451745184519452045214522452345244525452645274528452945304531453245334534453545364537453845394540454145424543454445454546454745484549455045514552455345544555455645574558455945604561456245634564456545664567456845694570457145724573457445754576457745784579458045814582458345844585458645874588458945904591459245934594459545964597459845994600460146024603460446054606460746084609461046114612461346144615461646174618461946204621462246234624

In [6]:
print(interactions.shape)
igindex=[]
for index, row in interactions.iterrows():
    if('Gene' not in row['preferredName_A'] or ('Gene' not in row['preferredName_B'])):
        igindex.append(index)
interactions = interactions.drop(labels=igindex, axis=0)
print(interactions.shape)
print("before cosine calc, the scores from STRING\n",interactions.head())
interactions.to_csv('string_interactions.csv', encoding='utf-8', index=False)

(6199, 3)
(5888, 3)
before cosine calc, the scores from STRING
   preferredName_A preferredName_B  score
0       Gene_1001       Gene_1002  0.569
1       Gene_1001       Gene_1002  0.569
2       Gene_1001       Gene_1000  0.575
3       Gene_1001       Gene_1000  0.575
4      Gene_10016      Gene_10015  0.998


In [7]:
import multiprocessing
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import os
import sys
import json
print(interactions.shape)
def load_embedding(path):
    embedding = KeyedVectors.load_word2vec_format(path, binary=True)
    print('embedding loaded from', path)
    return embedding
w2v_model = load_embedding("bioconceptvec_word2vec_cbow.bin")

def cosine(a, b):
    norm1 = np.linalg.norm(a)
    norm2 = np.linalg.norm(b)
    return np.dot(a, b) / (norm1 * norm2)
ignoreindex=[]
prediction=[]
for index, row in interactions.iterrows():
    if('Gene' not in row['preferredName_A'] or ('Gene' not in row['preferredName_B'])):
        ignoreindex.append(index)
    else:
        row['score']=cosine(w2v_model[row['preferredName_A']], w2v_model[row['preferredName_B']])
trueval=0
print("after cosine calc:",interactions.head())
for index, row in interactions.iterrows():
    if(index not in ignoreindex ):
        if(row['score']<0.5):
            prediction.append(0)
        else:
            prediction.append(1)
            trueval+=1
print(trueval) # 4192 predicted as true
print(len(ignoreindex)) # didn't have their entrenz
print(len(prediction)) # total 5888 . 4192 as true and remaining as false
            



(5888, 3)
before cosine calc, the scores from STRING   preferredName_A preferredName_B  score
0       Gene_1001       Gene_1002  0.569
1       Gene_1001       Gene_1002  0.569
2       Gene_1001       Gene_1000  0.575
3       Gene_1001       Gene_1000  0.575
4      Gene_10016      Gene_10015  0.998
embedding loaded from bioconceptvec_word2vec_cbow.bin
after cosine calc:   preferredName_A preferredName_B     score
0       Gene_1001       Gene_1002  0.577314
1       Gene_1001       Gene_1002  0.577314
2       Gene_1001       Gene_1000  0.537557
3       Gene_1001       Gene_1000  0.537557
4      Gene_10016      Gene_10015   0.45973
4192
0
5888


In [8]:
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
y_true=[1]*len(prediction)
print(precision_recall_fscore_support(y_true, prediction, average='macro'))

target_names = ['0', '1'] # 0 means no interation, 1 means identified as interaction as cosine score>0.5
print(classification_report(y_true, prediction, target_names=target_names))


(0.5, 0.35597826086956524, 0.4158730158730159, None)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       1.00      0.71      0.83      5888

    accuracy                           0.71      5888
   macro avg       0.50      0.36      0.42      5888
weighted avg       1.00      0.71      0.83      5888



C:\Users\Yoshita\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Yoshita\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Yoshita\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1308: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Yoshita\anaconda3\lib\site-packages\sklearn\metrics\_classificat